# Divulgação Farol v2

Temas para pauta:

**Novos casos nas capitais por 100 mil habitantes**
- Maior gravidade da doença em capitais fora do sudeste
- Nível de hoje de novo casos maior que em abril: não estamos no novo normal
- Explorar o que pode explicar a diferença nas capitais: expansão de leitos? Qual o nível de subnotificação?

**Distanciamento social**
- Hipótese: estamos voltando ao nível pré-pandemia
- Entender os recortes para usarmos (capitais x interior? regiões?)

**Níveis de Alerta nas capitais**

---

Atualizações:

- 1a versão: 22/09/2020
- 2a versão: 16/10/2020
- 3a versão: 24/11/2020

## Pacotes

In [1]:
import plotly.offline
import plotly.graph_objs as go
import cufflinks as cf

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

plotly.offline.init_notebook_mode(connected=True)
import pandas as pd
import numpy as np

import os
import yaml
import requests

## Código IBGE das capitais

In [2]:
ibge_capitais = {
    "Rio Branco": 1200401,
    "Maceió": 2704302,
    "Macapá": 1600303,
    "Manaus": 1302603,
    "Salvador": 2927408,
    "Fortaleza": 2304400,
    "Brasília": 5300108,
    "Vitória": 3205309,
    "Goiânia": 5208707,
    "São Luís": 2111300,
    "Cuiabá": 5103403,
    "Campo Grande": 5002704,
    "Belo Horizonte": 3106200,
    "Belém": 1501402,
    "João Pessoa": 2507507,
    "Curitiba": 4106902,
    "Recife": 2611606,
    "Teresina": 2211001,
    "Rio de Janeiro": 3304557,
    "Natal": 2408102,
    "Porto Alegre": 4314902,
    "Porto Velho": 1100205,
    "Boa Vista": 1400100,
    "Florianópolis": 4205407,
    "São Paulo": 3550308,
    "Aracaju": 2800308,
    "Palmas": 1721000
}

regions = {
    "Nordeste": [
        "SE",
        "CE",
        "PB",
        "AL",
        "RN",
        "PE",
        "BA",
        "MA",
        "PI",
    ],
    "Sudeste": [
        "MG",
        "RJ",
        "SP",
        "ES"
    ],
    "Norte": [
        "PA",
        "RR",
        "AP",
        "AM",
        "TO",
        "RO",
        "AC"
    ],
    "Centro-Oeste": [
        "DF",
        "MS",
        "MT",
        "GO"
    ],
    "Sul": [
        "PR",
        "SC",
        "RS"
    ]
}

flat_regions = dict()

for region in regions.keys():
    flat_regions.update({uf: region for uf in regions[region]})
    
flat_regions

{'SE': 'Nordeste',
 'CE': 'Nordeste',
 'PB': 'Nordeste',
 'AL': 'Nordeste',
 'RN': 'Nordeste',
 'PE': 'Nordeste',
 'BA': 'Nordeste',
 'MA': 'Nordeste',
 'PI': 'Nordeste',
 'MG': 'Sudeste',
 'RJ': 'Sudeste',
 'SP': 'Sudeste',
 'ES': 'Sudeste',
 'PA': 'Norte',
 'RR': 'Norte',
 'AP': 'Norte',
 'AM': 'Norte',
 'TO': 'Norte',
 'RO': 'Norte',
 'AC': 'Norte',
 'DF': 'Centro-Oeste',
 'MS': 'Centro-Oeste',
 'MT': 'Centro-Oeste',
 'GO': 'Centro-Oeste',
 'PR': 'Sul',
 'SC': 'Sul',
 'RS': 'Sul'}

## Novos casos nas capitais

- Capitais fora do sudeste apresentam maior incidência de casos por 100mil habitantes.

### Importa dados

In [3]:
day = 16
month = 11
year = 2020
analysis_date = f"{year}{month}{day}"

#### Baixar dados novamente

In [4]:
# Reload data from source
df = (pd.read_csv("http://datasource.coronacidades.org/br/cities/cases/full")
         .assign(last_updated=lambda df: pd.to_datetime(df["last_updated"])))

# Create New Data Directory if Necessary
new_data_path = "{}/data/{}".format(os.getcwd(), analysis_date)
if not os.path.exists(new_data_path):
    os.mkdir(new_data_path)

# Save capitals data to CSV
capitals = df[df["city_id"].isin(ibge_capitais.values())]
capitals.to_csv("data/{}/br_capitals_cases_{}.csv".format(analysis_date, analysis_date))

#### Carregar dados existentes

In [ ]:
capitals = (pd.read_csv(f"data/{analysis_date}/br_capitals_cases_{analysis_date}.csv", index_col=0)
           .assign(last_updated=lambda df: pd.to_datetime(df["last_updated"])))

capitals.info()

In [ ]:
capitals["last_updated"].max()

#### Remove datas com valores negativos (inconsistências de reporte)

- 4 datas com novos casos negativos

In [ ]:
capitals[capitals["daily_cases_mavg"] < 0][["last_updated", "city_id", "city_name", "population", 
                                            "daily_cases", "daily_cases_mavg", "daily_cases_mavg_100k"]]

In [ ]:
capitals = capitals[capitals["daily_cases_mavg"] > 0]

#### Filtra a partir de 27 de março (todas as capitais com casos)

In [ ]:
first_date = capitals.groupby("city_name")["last_updated"].min().max()
first_date

In [ ]:
capitals = capitals[capitals["last_updated"] > first_date]

In [ ]:
last_updated = capitals["last_updated"].max().strftime("%d/%m/%Y")
last_updated

### Distribuição de novos casos 

Gráficos:
- Média semanal de novos casos /100 mil hab. nas capitais no tempo
- Mapa de calor da média semanal de novos casos /100 mil hab. nas capitais no tempo
- Mapa de calor da média semanal de novos casos /100 mil hab. **em log** nas capitais no tempo
- Mapa de calor da média semanal de novos casos /100 mil hab. **normalizado** nas capitais no tempo

In [ ]:
title = f"Média semanal de novos casos /100 mil hab. nas capitais (até {last_updated})"
capitals.pivot_table(index="last_updated", 
                     columns="city_name", 
                     values="daily_cases_mavg_100k").iplot(kind="scatter", 
                                                           theme="white",
                                                           title=title)

In [ ]:
def plot_heatmap(df, title, colorscale="temps", is_reverse=False):
    data = go.Heatmap(
        {"z": df.values.tolist(), "x": df.columns.tolist(), "y": df.index.tolist()}, 
        colorscale=colorscale,
        showscale=True,
        reversescale=is_reverse
    )
    layout = go.Layout(
        title=title,
        height=800,
        margin={"l": 100, "r": 100, "t": 100},
        xaxis=dict(domain=[0, 0.8]),
        yaxis=dict(domain=[0, 1], tickmode="linear"),
        # yaxis2=dict(tickmode="linear", anchor="x2"),
    )

    return go.Figure(data=data, layout=layout)

### Heatmap absoluto

In [ ]:
# ordena pelo pico mais recente
pivot = capitals.pivot(index="city_name", columns="last_updated", values="daily_cases_mavg_100k")
# pivot = pivot.apply(lambda x: x / x.max(), axis=1)
pivot = pivot.loc[pivot.idxmax(axis=1).sort_values().index]

In [ ]:
title = f"Mapa de calor da média semanal de novos casos / 100 mil hab. nas capitais (até {last_updated})"
plot_heatmap(pivot, title=title)

### Tabela de comparação mais recente e máximo das capitais

In [ ]:
table = (capitals.loc[capitals.groupby("city_name")["last_updated"].idxmax()][
    [
        "city_name", 
        "state_id",
        "population", 
        "daily_cases_mavg_100k",
        "last_updated"
    ]
]
 .sort_values(["daily_cases_mavg_100k"], ascending=False)
 .set_index("city_name")
)

table = table.assign(region=table["state_id"].map(flat_regions))
table

In [ ]:
order = table.index.values
order

In [ ]:
# Maximo
table_max = pd.DataFrame(capitals.groupby("city_name")["daily_cases_mavg_100k"].max().loc[order])

# Data do maximo
table_max["last_updated"] = capitals.set_index("last_updated").groupby("city_name")["daily_cases_mavg_100k"].idxmax().loc[order]

table_compare = (table.join(table_max, rsuffix="_max")
                 .assign(rank= lambda df : df["daily_cases_mavg_100k"].rank(ascending=False))
                 .sort_values("daily_cases_mavg_100k", ascending=False)
                 .reset_index())

table_cols = {
    "rank": "Ranking",
    "city_name": "Capital",
    "state_id": "Estado",
    "region": "Região",
    "population": "População",
    "daily_cases_mavg_100k": "Média de casos por 100mil hab. (+ recente)",
    "daily_cases_mavg_100k_max": "Média de casos por 100mil hab. (máximo)",
    "last_updated": "Data mais recente",
    "last_updated_max": "Data do máximo (pico)"
}

table_compare = table_compare.rename(columns=table_cols)[table_cols.values()]
table_compare

#### Gravar dados de casos por 100 mil habitantes em capitais

In [ ]:
table_compare.to_csv("data/{}/br_capitals_daily_cases_mavg_100k_last_{}.csv".format(analysis_date, analysis_date))

### Heatmap em escala log10

In [ ]:
capitals["daily_cases_mavg_100k_log"] = capitals["daily_cases_mavg_100k"].apply(lambda x: np.log10(x+1))

# ordena pelo pico mais recente
pivot = capitals.pivot(index="city_name", columns="last_updated", values="daily_cases_mavg_100k_log")
# pivot = pivot.apply(lambda x: x / x.max(), axis=1)
pivot = pivot.loc[pivot.idxmax(axis=1).sort_values().index]

title = f"""
Mapa de calor da média semanal de novos casos / 100 mil hab.
nas capitais (até {last_updated})"""

plot_heatmap(pivot, title=title)

In [ ]:
order = pivot.index
order

## Distanciamento social nas capitais

#### Baixar dados novamente

In [ ]:
# Reload data from source
df_inloco = (pd.read_csv("http://datasource.coronacidades.org/inloco/cities/ufVhdASsYeoxVgcg7Fvih8J3HnbZPHbp")
             .assign(last_updated=lambda df: pd.to_datetime(df["dt"])))

# Create New Data Directory if Necessary
new_data_path = "{}/data/{}".format(os.getcwd(), analysis_date)
if not os.path.exists(new_data_path):
    os.mkdir(new_data_path)

# Save capitals inloco data to CSV
capitals_inloco = df_inloco[df_inloco["city_id"].isin(ibge_capitais.values())].dropna()
capitals_inloco.to_csv("data/{}/br_capitals_inloco_{}.csv".format(analysis_date, analysis_date))
capitals_inloco.info()

#### Carregar dados existentes

In [ ]:
capitals_inloco = (pd.read_csv(f"data/{analysis_date}/br_capitals_inloco_{analysis_date}.csv", index_col=0)
                  .assign(last_updated=lambda df: pd.to_datetime(df["dt"])))
capitals_inloco.info()

In [ ]:
last_updated_inloco = capitals_inloco["last_updated"].max().strftime("%d/%m/%y")
last_updated_inloco

### Distribuição e valores da série

In [ ]:
data = pd.DataFrame(index=capitals_inloco[["city_name"]].drop_duplicates().city_name)

for i in range(2, 11):
    data[f"2020-0{i}"] = capitals_inloco[capitals_inloco["last_updated"].dt.month == i].groupby("city_name").mean()[["isolated"]]

100*data.mean()

In [ ]:
(100*data).style.background_gradient(cmap='Greens')

#### Gravar Dados de Médias

In [ ]:
(100*data).to_csv("data/{}/br_capitals_inloco_months.csv".format(analysis_date))

### Comparação com série de novos casos

In [ ]:
compare_cases_inloco = capitals_inloco.sort_values(["city_name", "last_updated"]).reset_index()
# compare_case_inloco = compare_cases_inloco[capitals_inloco["last_updated"] > first_date]
compare_cases_inloco["isolated_mavg"] = 10*compare_cases_inloco.groupby("city_name").rolling(7,7,on="last_updated")["isolated"].mean().reset_index(drop=True)

In [ ]:
compare_cases_inloco.info()

In [ ]:
# ordena pelo pico mais recente
pivot = compare_cases_inloco.pivot(index="city_name", columns="last_updated", values="isolated_mavg")
# pivot = pivot.apply(lambda x: x / x.max(), axis=1)
pivot = pivot.loc[order]

title = f"Mapa de calor da média móvel do distanciamento social nas capitais (até {last_updated_inloco})"
plot_heatmap(pivot, title=title, is_reverse=True)

## Níveis de Alerta

#### Baixar Dados Novamente

In [ ]:
# Reload config data from source and save to CSV
url = "https://raw.githubusercontent.com/ImpulsoGov/farolcovid/farol-v2/src/configs/config.yaml"

config = yaml.load(requests.get(url).text, Loader=yaml.FullLoader)

with open('data/config.yml', 'w') as yaml_file:
    yaml.dump(config, yaml_file, default_flow_style=False)

In [ ]:
# Reload alert level data from source
df_alert = (pd.read_csv("http://datasource.coronacidades.org/br/cities/farolcovid/main")
            .assign(
                last_updated_cases=lambda df: pd.to_datetime(df["last_updated_cases"]),
                overall_classification=lambda df: df["overall_alert"].map(config["br"]["farolcovid"]["categories"])))

# Create New Data Directory if Necessary
new_data_path = "{}/data/{}".format(os.getcwd(), analysis_date)
if not os.path.exists(new_data_path):
    os.mkdir(new_data_path)

# Save alert level data to CSV
df_alert.to_csv("data/{}/br_cities_farolcovid_main_{}.csv".format(analysis_date, analysis_date))

#### Carregar Dados Existentes

In [ ]:
config = yaml.load(open('data/config.yml', 'r'), Loader=yaml.FullLoader)
colors = dict(zip(config["br"]["farolcovid"]["categories"].values(), 
                  ["#0990A7", "#F7B502", "#F77800", "#F22E3E"]))
colors

In [ ]:
df_alert = (pd.read_csv(f"data/{analysis_date}/br_cities_farolcovid_main_{analysis_date}.csv", index_col=0)
            .assign(last_updated_cases=lambda df: pd.to_datetime(df["last_updated_cases"])))

df_alert.info()

In [ ]:
df_alert["last_updated_cases"].max()

### Níveis nas capitais

In [ ]:
capitals_alert = df_alert[df_alert["city_id"].isin(ibge_capitais.values())]

In [ ]:
capitals_alert[["state_id", "city_id"]].drop_duplicates().set_index("city_id")

In [ ]:
alert_cols = {
    "city_name": "Capital",
    "state_id": "Estado",
    "region": "Região",
    "situation_classification": "SITUAÇÃO",
    "control_classification": "CONTROLE",
    "capacity_classification": "CAPACIDADE",
    "trust_classification": "CONFIANÇA",
    "overall_classification": "NÍVEL DE ALERTA",
}

In [ ]:
col_classif = [i for i in df_alert if "classification" in i]

capitals_alert_classifications = (
    capitals_alert
    .assign(region= lambda df: df["state_id"].map(flat_regions))
    .set_index(["city_name", "state_id", "region"])
    .loc[reversed(order)][col_classif]
    .replace(config["br"]["farolcovid"]["categories"])
    .reset_index()
)

capitals_alert_classifications = capitals_alert_classifications.rename(columns=alert_cols)[alert_cols.values()]
capitals_alert_classifications

#### Gravar dados de níveis de alerta de capitais

In [ ]:
#capitals_alert_classifications.to_csv(f"data/{analysis_date}/br_capitals_indicators_alert_{analysis_date}.csv")

### Taxa de subnotificação nas capitais

#### Capitais em alerta Altíssimo

In [ ]:
capitals_alert[capitals_alert["overall_classification"] == "altíssimo"]["subnotification_rate"].describe()

#### Demais capitais

In [ ]:
capitals_alert[capitals_alert["overall_classification"] != "altíssimo"]["subnotification_rate"].describe()

#### Gravar dados de níveis de alerta de capitais

In [ ]:
capitals_alert_classifications.to_csv(f"data/{analysis_date}/br_capitals_indicators_alert_{analysis_date}.csv")

### Níveis nas cidades por estado

In [ ]:
# calcula percentual de cidades em cada nivel de alerta por estado
states_alert_n_cities = (df_alert.pivot_table(index="state_id", 
                     columns="overall_classification", 
                     values="city_id", aggfunc="count")
 .fillna(0).sort_values("altíssimo", ascending=False)
 .apply(lambda row: row/row.sum(), axis=1))

# add total de cidades no estado
states_alert_n_cities = states_alert_n_cities.join(df_alert.groupby(["state_id"]).count()[["city_id"]])
states_alert_n_cities

#### Gravar dados para demais cidades no estado

In [ ]:
states_alert_n_cities.to_csv(f"data/{analysis_date}/br_states_alert_n_cities_{analysis_date}.csv")